<a href="https://colab.research.google.com/github/elenancalima/Troph_Min_5to2/blob/main/YOLOv11n_seg_Training_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup:


In [1]:


# Step 1: Install the ultralytics library
# We use 'pip' to install the package that contains the YOLO models and training logic.
# The '!' prefix allows us to run shell commands directly in the notebook.
print("Installing the 'ultralytics' library...")
!pip install ultralytics

# Import the main YOLO class from the ultralytics library
from ultralytics import YOLO
import torch

# Step 2: Verify GPU availability
# This is a crucial step to ensure that the training will be accelerated.
# If a GPU is not available, training will be extremely slow.
print("\nVerifying GPU availability...")
if torch.cuda.is_available():
    print(f"GPU is available. Using device: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available. Training will run on CPU, which may be very slow.")

# Step 3: Load a pretrained YOLO model
# It's highly recommended to start with a pretrained model ('*.pt' file),
# as it has already learned basic features from a large dataset (like COCO).
# This process is called transfer learning.
model = YOLO("yolo11s-seg.pt")  # load a pretrained segmentation model

# --- Alternative ways to load a model (for reference) ---
# model = YOLO("yolo11n-seg.yaml")  # build a new model from a YAML configuration
# model = YOLO("yolo11n-seg.yaml").load("yolo11n.pt")  # build from YAML and transfer weights

Installing the 'ultralytics' library...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.4 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

Verifying GPU availability...
GPU is available. Using device: Tesla T4


Dataset:

In [2]:
# This command connects your Colab notebook to your Google Drive.
# You will be prompted to authorize access.
from google.colab import drive
drive.mount('/content/drive')

# Now, unzip your dataset from your Drive into the main Colab workspace.
# Replace 'your_dataset_name.zip' with the actual name of your file.
# This places your data in /content/your_dataset_name/
#!unzip /content/drive/MyDrive/sample_yolo_dataset.zip -d /content/

!unzip /content/drive/MyDrive/tag_occluded_dataset.zip -d /content/

Mounted at /content/drive
Archive:  /content/drive/MyDrive/tag_occluded_dataset.zip
   creating: /content/dataset/
   creating: /content/dataset/images/
   creating: /content/dataset/images/train/
  inflating: /content/dataset/images/train/ManyAnts2.5D_gR0030_feeding_04197.png  
  inflating: /content/dataset/images/train/ManyAnts2.5D_gR0042_feeding_02960_crop.png  
  inflating: /content/dataset/images/train/ManyAnts2.5D_gR0049_feeding_03100.png  
  inflating: /content/dataset/images/train/ManyAnts2.5D_gR0069_feeding_04001.png  
   creating: /content/dataset/images/val/
  inflating: /content/dataset/images/val/ManyAnts2.5D_gR0073_feeding_03866_crop.png  
   creating: /content/dataset/labels/
  inflating: /content/dataset/labels/ManyAnts2.5D_gR0030_feeding_04197.txt  
  inflating: /content/dataset/labels/ManyAnts2.5D_gR0042_feeding_02960_crop.txt  
  inflating: /content/dataset/labels/ManyAnts2.5D_gR0049_feeding_03100.txt  
  inflating: /content/dataset/labels/ManyAnts2.5D_gR0069_feeding

In [3]:

!cat /content/my_dataset.yaml

!ls /content/dataset/images/train/
!ls -l /content/dataset/labels/train/


!rm /content/dataset/labels/val.cache
!rm /content/dataset/labels/train.cache

cat: /content/my_dataset.yaml: No such file or directory
ManyAnts2.5D_gR0030_feeding_04197.png
ManyAnts2.5D_gR0042_feeding_02960_crop.png
ManyAnts2.5D_gR0049_feeding_03100.png
ManyAnts2.5D_gR0069_feeding_04001.png
total 16
-rw-rw-rw- 1 root root 2428 Oct 14 17:16 ManyAnts2.5D_gR0030_feeding_04197.txt
-rw-rw-rw- 1 root root 2464 Oct 14 17:16 ManyAnts2.5D_gR0042_feeding_02960_crop.txt
-rw-rw-rw- 1 root root 2041 Oct 14 17:16 ManyAnts2.5D_gR0049_feeding_03100.txt
-rw-rw-rw- 1 root root 1995 Oct 14 17:16 ManyAnts2.5D_gR0069_feeding_04001.txt
rm: cannot remove '/content/dataset/labels/val.cache': No such file or directory
rm: cannot remove '/content/dataset/labels/train.cache': No such file or directory


In [4]:
%%writefile /content/my_dataset.yaml
# Path to the dataset directory inside the Google Colab environment
path: /content/dataset

# Directories for training and validation images (relative to the path above)
train: images/train
val: images/val

# A list of your class names.
names:
  0: tag # Or whatever your single class is named
  #1: headthorax

Writing /content/my_dataset.yaml


In [5]:
from google.colab import drive

drive.mount('/content/drive')  # authorizes Drive
save_root = "/content/drive/MyDrive/yolo_runs"  # pick any folder in Drive
run_name = "y11n_seg_exp1_tag"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


First training:

Mind run name!!


In [6]:

print("\nStarting model training...")
#results = model.train(data="coco8-seg.yaml", epochs=5, imgsz=640)
results = model.train(data="/content/my_dataset.yaml", epochs=600, imgsz=640,
                      project=save_root,          # <- Drive path
                      name=run_name,       # subfolder name
                      exist_ok=True)               # allow reuse of the folder)


# Step 5: Print a confirmation message upon completion
print("\nTraining complete!")
print("You can view the training results, including saved model weights and performance charts, in the 'runs' directory.")


Starting model training...
Ultralytics 8.3.218 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/my_dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=600, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=y11n_seg_exp1_tag, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=

KeyboardInterrupt: 

Continue training (mind the train dir):

attempt 1

Low-threshold visualization (check train dir):

In [ ]:
# Load your best-trained model
from ultralytics import YOLO
from pathlib import Path

best_path = Path(save_root) / run_name / "weights" / "best.pt"
model = YOLO(str(best_path))

# Find the path to your validation image
# This assumes your YAML file is at '/content/my_dataset.yaml'
# and your validation images are in '/content/your_dataset_name/images/val/'
validation_image_path = '/content/dataset/images/val/ManyAnts2.5D_gR0073_feeding_03866_crop.png'

# Run prediction with a very low confidence threshold (e.g., 0.01)
# The 'save=True' argument will save the output image with overlays.

results = model.predict(
    source=validation_image_path,
    conf=0.7,
    retina_masks=True,
    save=True,
    save_txt=True,     # writes labels to runs/predict*/labels/*.txt
    save_conf=True     # includes confidence in each line
)



image 1/1 /content/dataset/images/val/ManyAnts2.5D_gR0073_feeding_03866_crop.png: 640x640 29 tags, 20.1ms
Speed: 1.6ms preprocess, 20.1ms inference, 4.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/runs/segment/predict4
1 label saved to /content/runs/segment/predict4/labels
